In [1]:
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from skimage import exposure

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_handle = "./magenta_arbitrary-image-stylization-v1-256_2"
hub_module = hub.load(hub_handle)

EXTERNAL = True

if EXTERNAL:
  %store -r WIDTH
  %store -r HEIGHT
  %store -r TEXT
  %store -r VISUALIZE
  %store -r SAVE_FOLDER
  %store -r PROJECT_NAME
  %store -r SHOW_OUTPUT
  %store -r IMG_NUMBER
  %store -r THUMBNAIL
 

else:
  WIDTH = 768
  HEIGHT = 768
  VISUALIZE = True  
  SAVE_FOLDER = "D:\\Deletar\\p_gen"
  PROJECT_NAME = "THE FELLOWSHIP OF THE RING"
  STYLES_FOLDER = ".\\styles"
  SHOW_OUTPUT = True
  IMG_NUMBER = 2
  
  

project_folder = f"{SAVE_FOLDER}\\{PROJECT_NAME}"

In [2]:
def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image


""" def load_image(image_url, image_size=(256, 256),
preserve_aspect_ratio=True): """
@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size):
  """Loads and preprocesses images."""
  # Cache image file locally.
  """ image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url) """
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(image_url),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  """ img = crop_center(img) """
  img = tf.image.resize(img, image_size, antialias=True)
  return img

# @title Load example images  { display-mode: "form" }


# The content image size can be arbitrary.

# The style prediction model was trained with image size 256 and it's the 
# recommended image size for the style image (though, other sizes work as 
# well but will lead to different results).
style_img_size = (256, 256)  # Recommended to keep it at 256.


In [3]:
style_img_mod = []
style_name = []
for img in os.listdir(STYLES_FOLDER):
    style_name.append(img)
    temp_img = load_image(f"{STYLES_FOLDER}\\{img}", style_img_size)
    style_img_mod.append(tf.nn.avg_pool(temp_img, ksize=[3,3], strides=[1,1], padding='SAME'))

In [ ]:
cont = 0
if EXTERNAL:
    if (not os.path.exists(f"{project_folder}\\style\\")):
        os.makedirs(f"{project_folder}\\style\\")
    for folder_name in os.listdir(os.path.join(f"{project_folder}\\img\\")):
        print(folder_name)
        for filename in os.listdir(os.path.join(f"{project_folder}\\img\\{folder_name}")):
            if (not os.path.exists(f"{project_folder}\\style\\{folder_name}")):
                    os.makedirs(f"{project_folder}\\style\\{folder_name}")
            if (filename == "c.jpg"):
                content_image = load_image(os.path.join(f"{project_folder}\\img\\{folder_name}\\{filename}"), (HEIGHT, WIDTH))
                tf.keras.preprocessing.image.save_img(f"{project_folder}\\style\\{folder_name}\\c.jpg",content_image[0])
                for i, dif_style in enumerate(style_img_mod):
                    outputs = hub_module(tf.constant(content_image), tf.constant(dif_style))
                    stylized_image = outputs[0][0]
                    
                    # stylized_image = exposure.equalize_hist(stylized_image*255)*255                    
                    tf.keras.preprocessing.image.save_img(f"{project_folder}\\style\\{folder_name}\\result_a{i}.jpg",stylized_image)
else:
    content_image_url = 'D:\\OneDrive\\Desktop\\machine-2\\c.jpg'  # @param {type:"string"}
    content_image = load_image(content_image_url, (WIDTH, HEIGHT))

    for i, dif_style in enumerate(style_img_mod):
        print(style_name[i])
        outputs = hub_module(tf.constant(content_image), tf.constant(dif_style))
        stylized_image = outputs[0][0]    
        tf.keras.preprocessing.image.save_img(f"result_a{i}.jpg",stylized_image)
        plt.imshow(stylized_image) 
        plt.show()
